# Generate Dataset
using the raw data in database, we will generate dataset using DatasetGenerator

In [1]:
from datetime import datetime

from dataset.Labeler import Labeler
from dataset.dataset_generator import DatasetGenerator
from dataset.feature_transformer import FeatureTransformer
from dataset.session_filterer import SessionFilterer

In [3]:
dataset_generator = DatasetGenerator(
    feature_transformer=FeatureTransformer(),
    labeler=Labeler(),
    session_filterer=SessionFilterer(),

    start_date=datetime(2023, 3, 1), end_date=datetime(2023, 4, 1)
)

X, Y = dataset_generator.generate()

generate dataset...


128778it [00:11, 11039.02it/s]


generate dataset done.


In [4]:
X.shape, Y.shape

((104702, 4), (104702,))

array([0.60201667, 0.56325   , 0.39651667, 0.57905   , 6.11645   ,
       5.95556667, 5.83566667, 5.71813333, 5.65556667, 5.40785   ])